In [31]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 상가 면적 (pnu단위_19자리)
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']
df_f_tot = df_f_tot[~df_f_tot['f_tot'].isnull()].reset_index(drop=True)

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 상가면적 / 거주인구
df['population_per_commercial'] = (df['f_tot'] / df['total_count'])
df = df[['emd_cd','population_per_commercial']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_population_per_commercial'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_population_per_commercial;
    create table m2.datadam_population_per_commercial (    
    emd_cd varchar(10)
    , population_per_commercial double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 19:26:35.944487  :  0
2022-10-30 19:26:38.039882  :  10000
17296


In [30]:
df

,emd_cd,population_per_commercial,base_ym
0,1111010100,71464.562500,202210
1,1111010200,132057.057143,202210
2,1111010300,25740.906977,202210
3,1111010600,362829.345455,202210
4,1111011000,664140.230769,202210
...,...,...,...
18597,4889039030,4.606061,202210
18598,4889041026,41.153846,202210
18599,4889041030,11.597015,202210
18600,4889042022,2.032258,202210


In [32]:
df.describe()

,population_per_commercial
count,17296.000000
mean,45176.698724
std,153540.963842
min,0.058480
25%,49.351316
50%,321.275980
75%,3864.073727
max,3469255.572881


In [27]:
from sklearn.preprocessing import RobustScaler

# 변형 객체 생성
robust_scaler = RobustScaler()

# 훈련데이터의 모수 분포 저장
robust_scaler.fit(df)

# 훈련 데이터 스케일링
df_scaled = robust_scaler.transform(df)

In [28]:
df_scaled

array([[-7.65643687, 18.64966277,  0.        ],
       [-7.65643665, 34.53351698,  0.        ],
       [-7.65643642,  6.66355982,  0.        ],
       ...,
       [ 0.83376818, -0.08117995,  0.        ],
       [ 0.83377041, -0.08368727,  0.        ],
       [ 0.83377268, -0.07886248,  0.        ]])